In [7]:
%pip uninstall hopsworks hsfs hsml -y

%pip install hopsworks

Found existing installation: hopsworks 4.1.4
Uninstalling hopsworks-4.1.4:
  Successfully uninstalled hopsworks-4.1.4
Note: you may need to restart the kernel to use updated packages.
  Using cached hopsworks-4.1.4-py3-none-any.whl.metadata (11 kB)
Using cached hopsworks-4.1.4-py3-none-any.whl (640 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
import os

In [11]:
project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

feature_view = fs.get_feature_view(
    name='tmdb_fv',
    version=2,
)

2025-01-05 20:56:12,976 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 20:56:12,985 INFO: Initializing external client
2025-01-05 20:56:12,985 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 20:56:14,400 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559


In [12]:
mr= project.get_model_registry()

retrieved_model=mr.get_model(
    name="movie_rating_model",
    version=1,
)

saved_model_dir=retrieved_model.download()

In [13]:
retrieved_xgboost_model=XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir+"/model.json")

retrieved_xgboost_model

XGBRegressor(base_score='6.0506077E0', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['int', 'float', 'float', 'float', 'int', 'int',
                            'int', 'int', 'int', 'int', 'int', 'int', 'int',
                            'int', 'int', 'int', 'int', 'int', 'in...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [27]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alanvourch/tmdb-movies-daily-updates")

# List files in the downloaded dataset directory
files = os.listdir(path)
print("Files in dataset directory:", files)

# Read the first file
updated_df = pd.read_csv(os.path.join(path, files[0]))

# Print the first 5 rows
updated_df.head()

Files in dataset directory: ['TMDB_all_movies.csv']


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
0,2,Ariel,7.1,335.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,...,suomi,"Heikki Salomaa, Hanna Jokinen, Matti Pellonpää...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN,7.4,8868.0,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,Shadows in Paradise,7.3,400.0,Released,1986-10-17,0.0,74.0,0.0,tt0092149,...,"English, suomi, svenska","Teuvo Rissanen, Malla Hukkanen, Olli Varja, Ju...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN,7.5,7646.0,/nj01hspawPof0mJmlgfjuLyJuRN.jpg
2,5,Four Rooms,5.9,2650.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,English,"Lili Taylor, Valeria Golino, Marisa Tomei, Mar...","Quentin Tarantino, Robert Rodriguez, Allison A...","Andrzej Sekula, Rodrigo García, Guillermo Nava...","Quentin Tarantino, Robert Rodriguez, Allison A...","Quentin Tarantino, Lawrence Bender, Alexandre ...",Combustible Edison,6.7,113064.0,/pyCk5JgtRZwRxnXwfrvyzukaKue.jpg
3,6,Judgment Night,6.5,333.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,...,English,"Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Marilyn Vance, Lloyd Segan",Alan Silvestri,6.6,19437.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
4,8,Life in Loops (A Megacities RMX),7.5,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,...,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN,8.2,284.0,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg


In [28]:
updated_df['status'].value_counts()

status
Released           1021151
Planned               5278
Post Production       5189
In Production         5028
Rumored               1090
Canceled               596
Name: count, dtype: int64

In [43]:
df = updated_df.copy()
df_new=df[df['status'].isin(['Post Production'])]
df_new

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
4032,8317,Tonight at Noon,0.0,0.0,Post Production,NaN,0.0,0.0,1000000.0,tt4032552,...,English,"Lauren Ambrose, Sasha Eden, Joan Chen, Ethan H...",Michael Almereyda,Scott Miller,"Jonathan Lethem, Michael Almereyda","Caldecot Chubb, Donald Rosenfeld, Emanuel Mich...",NaN,NaN,NaN,NaN
28123,43217,Coexist,4.8,2.0,Post Production,NaN,0.0,53.0,90000.0,tt1660327,...,"English, Kinyarwanda","Domitilie, Clémentine Mukaruziga, Agnes",Adam Mazo,NaN,NaN,"Suzanne Summerlin Tzuanos, Robert Koenig, Adam...",Adam Payne,8.5,10.0,NaN
34138,50500,Hotel Caledonia,0.0,0.0,Post Production,NaN,0.0,97.0,0.0,tt0913393,...,NaN,"James Cosmo, Kris Marshall, Leslie Phillips, J...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56141,79355,Arjunan Kadhali,0.0,0.0,Post Production,NaN,0.0,0.0,0.0,NaN,...,தமிழ்,"Kalaiyarasan, Lakshmi Ramakrishnan, Suzane Geo...",Parthy Baskar,NaN,"Kabilan, Parthy Baskar",Sivasakthi Pandian,NaN,NaN,NaN,/gx0I5AYlR69ffdUmynYxNZ3OAPY.jpg
59424,83533,Avatar: Fire and Ash,0.0,0.0,Post Production,2025-12-17,0.0,0.0,400000000.0,tt1757678,...,English,"Jemaine Clement, Jeremy Irwin, Brendan Cowell,...",James Cameron,Russell Carpenter,"Josh Friedman, Shane Salerno, James Cameron, R...","Jon Landau, James Cameron, Peter M. Tobyansen",Simon Franglen,NaN,NaN,/vDdFxksDMkUJjZXd00OKJfwXKre.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037844,1412075,民俗怪闻录之出马仙,0.0,0.0,Post Production,2025-01-09,0.0,0.0,0.0,NaN,...,普通话,"Cong Sihan, Liu Xinyu, Xie Shixin, Chen Chuang...",Xie Shaofeng,NaN,NaN,NaN,NaN,NaN,NaN,/3qoC35WCt4YgCQoUVWlT7Z0uRbi.jpg
1037913,1412173,Мой друг КотоПушкин,0.0,0.0,Post Production,NaN,0.0,0.0,0.0,tt32829065,...,Pусский,"Anya Izotova, Vera Biryukova, Leon Tafara, Nik...",Alena Mikhaylova,Anastasia Morozova,"Elena Sumina, Dmitry Medvedev","Anastasiya Biryukova, Maksim Koroptsov, Dmitry...",NaN,NaN,NaN,NaN
1037954,1412230,Disco Boy,0.0,0.0,Post Production,2025-01-18,0.0,19.0,0.0,NaN,...,NaN,"Angus Grinstead, Leo Odgers, Sukey Moschini Ridge",Mafalda Lorijn,"Jude Rodgers, Archie Bigg-Wither",Mafalda Lorijn,"Erin Davey, Mafalda Lorijn","Sebastian Hopton, Sophia Grover",NaN,NaN,/rcecxjfhofrsCseoKqvpHEswFiL.jpg
1038017,1412319,"Nothing, Maybe.",0.0,0.0,Post Production,NaN,0.0,15.0,0.0,tt33305403,...,עִבְרִית,"Tomer Barash, Ori Mechraz, Gil Seri, Omer Pere...",Daniel Gat,Shay Ater,Daniel Gat,Tamar Peled,NaN,NaN,NaN,NaN


In [47]:
# Have budget
df_b = df_new[(df_new['budget'] > 10000000.0)]

#Include year in 
year=2025
df_b["release_year"]=2025
df_b["imdb_votes"]=400
#Have votes? or pretend votes??
# Drop rows with NaN values in a specific column
#df_a = df_b.dropna(subset=['imdb_votes'])

# If you want to drop NaN values only in the specific column, and keep other columns unaffected, the above method works.

#print(df_planned['imdb_votes'].isnull().sum())

df_b.columns

2025-01-05 21:34:24,111 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-01-05 21:34:24,111 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'cast', 'director', 'director_of_photography', 'writers', 'producers',
       'music_composer', 'imdb_rating', 'imdb_votes', 'poster_path',
       'release_year'],
      dtype='object')

In [56]:
df_encoded = df_b.copy()
df_encoded = df_encoded[["runtime", "release_year", "imdb_votes", "budget", "genres"]]
df_encoded['genres'] = df_encoded['genres'].str.split(', ')  # Split by comma and space
genres_encoded = df_encoded['genres'].explode()  # Flatten the list
one_hot = pd.get_dummies(genres_encoded).groupby(level=0).sum()

one_hot.columns = one_hot.columns.str.lower()  # Lowercase column names
one_hot.columns = one_hot.columns.str.replace(' ', '_')  # Replace spaces with underscores

# Add missing columns if not already in the DataFrame
all_genres=['action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 'romance', 'science_fiction', 'tv_movie', 'thriller', 'war', 'western']
for col in all_genres:
        if col not in one_hot.columns:
            one_hot[col] = 0  # or df_encoded[col] = False if they were boolean flags

# Merge one-hot encoding with the original DataFrame
train_features = pd.concat([df_encoded, one_hot], axis=1)
train_features=train_features.drop(["genres"], axis=1)

# Ensure the column order matches the expected order
expected_columns = ['release_year', 'runtime', 'budget', 'imdb_votes', 'action', 'adventure', 'animation', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'mystery', 'romance', 'science_fiction', 'tv_movie', 'thriller', 'war', 'western']

train_features=train_features[expected_columns]

retrieved_xgboost_model.predict(train_features)

array([6.0760064, 5.8289905, 6.357131 , 5.5663886, 6.6046   , 6.0260696,
       6.2841415, 5.488776 , 6.654183 , 4.861762 , 6.8672705, 5.5165358,
       5.4702764, 6.3542147, 6.0581064, 4.866194 , 7.243296 , 6.4643836,
       5.8464355, 6.514565 , 5.285479 , 5.357504 , 6.679023 , 6.5732675,
       6.24667  , 6.0432477, 4.5275664, 7.1736307, 4.199864 , 6.686011 ,
       5.8747673, 7.4449496, 4.213533 , 6.9704056, 5.25238  , 4.6842885,
       4.986989 , 6.135772 , 5.261628 , 6.300657 , 6.3248854, 6.954775 ],
      dtype=float32)